<a href="https://colab.research.google.com/github/Avinash-sr/GenAI-collab-notebooks/blob/main/Rag_based_chatbot_with_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try :
  !pip install -q langchain openai tiktoken rapidocr-onnxruntime langchain_community wikipedia faiss-cpu
except Exception as e:
  print(e)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [3]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
from langchain.chat_models import ChatOpenAI
llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")

<ipython-input-4-090152d7e029>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")


In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

<ipython-input-5-d84b10cb3283>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [6]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import MessagesPlaceholder


In [7]:
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)
doc = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(doc)
vectorstore=FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

In [11]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

# chat_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )

# question_answering_chain=create_stuff_documents_chain(llm_model, chat_prompt)
# rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [12]:
# response = rag_chain.invoke({"input":"what is MRKL?"})


In [13]:
# response["answer"]

'MRKL, which stands for "Modular Reasoning, Knowledge, and Language," is a neuro-symbolic architecture for autonomous agents. It consists of expert modules and a general-purpose LLM that routes inquiries to the most suitable expert module, which can be neural or symbolic in nature. MRKL systems are designed to integrate neural (e.g., deep learning models) and symbolic (e.g., math calculators, weather APIs) components for various tasks.'

In [14]:
from langchain.chains import create_history_aware_retriever

retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

history_aware_retriever = create_history_aware_retriever(llm_model,retriever,contextualize_q_prompt)


In [15]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm_model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [17]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [18]:
conversational_rag_chain.invoke(
    {"input": "Hello My name is Avi I want to ask What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is a process where complex tasks are broken down into smaller and simpler steps to make them more manageable. Techniques like Chain of Thought (CoT) and Tree of Thoughts are used to help agents decompose hard tasks into multiple manageable tasks by thinking step by step or exploring multiple reasoning possibilities at each step. Task decomposition can be performed by AI models using simple prompting, task-specific instructions, or human inputs.'

In [19]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Task decomposition can be done using various methods, such as prompting AI models with simple instructions like "Steps for XYZ" or "What are the subgoals for achieving XYZ?", providing task-specific instructions like "Write a story outline" for writing a novel, or incorporating human inputs to break down complex tasks into smaller steps. Additionally, techniques like Chain of Thought (CoT) and Tree of Thoughts are commonly used to decompose tasks into multiple manageable subtasks.'

In [21]:
conversational_rag_chain.invoke(
    {"input": "My name is Avi"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Got it, Avi! How can I assist you today?'

In [22]:
conversational_rag_chain.invoke(
    {"input": "waht is my name"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Your name is Avi.'

In [23]:
conversational_rag_chain.invoke(
    {"input": "waht is my name"},
    config={"configurable": {"session_id": "abc23"}},
)["answer"]

"I don't know the answer to that question."